# 1. Imdb 다시 손코딩 해보기

## 1.1 conda env 자동 활성화

### 1.1.0 새로운 환경 설정

```
$ conda create -n algolab
```

### 1.1.1 autoenv 설치

참고 링크 : https://github.com/hyperupcall/autoenv

```
$ brew install 'autoenv'
```

### 1.1.2 자동으로 실행될 명령어 추가

```
$ echo "conda activate alogolab" >.env
```

### 1.1.3 해당 경로에서 터미널을 열면 자동으로 실행됨

```
CondaError: Run 'conda init' before 'conda activate'
```
위 에러가 나오지만 무시해도 됨

### 1.1.4 시행착오1 direnv 설치

참고 링크 : https://u2pia.medium.com/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EB%94%94%EB%A0%89%ED%86%A0%EB%A6%AC-%EB%93%A4%EC%96%B4%EA%B0%80%EB%A9%B4-%EC%9E%90%EB%8F%99%EC%9C%BC%EB%A1%9C-conda-%ED%99%98%EA%B2%BD-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0-direnv-7ed616ccea68

이 내용을 토대로 설치시 모든 경로에서 같은 env가 적용되는 문제가 있었음

### 1.1.5 시행착오2 WARNING 문구 발생

[WARNING]: Console output during zsh initialization detected.

When using Powerlevel10k with instant prompt, console output during zsh
initialization may indicate issues.

You can:

  - Recommended: Change ~/.zshrc so that it does not perform console I/O
    after the instant prompt preamble. See the link below for details.

    * You will not see this error message again.
    * Zsh will start quickly and prompt will update smoothly.

  - Suppress this warning either by running p10k configure or by manually
    defining the following parameter:

      typeset -g POWERLEVEL9K_INSTANT_PROMPT=quiet

    * You will not see this error message again.
    * Zsh will start quickly but prompt will jump down after initialization.

  - Disable instant prompt either by running p10k configure or by manually
    defining the following parameter:

      typeset -g POWERLEVEL9K_INSTANT_PROMPT=off

    * You will not see this error message again.
    * Zsh will start slowly.

  - Do nothing.

    * You will see this error message every time you start zsh.
    * Zsh will start quickly but prompt will jump down after initialization.

For details, see:
https://github.com/romkatv/powerlevel10k/blob/master/README.md#instant-prompt

-- console output produced during zsh initialization follows --

#### 해결법 : .zshrc 파일에 아래 내용을 최하단에 추가

```
typeset -g POWERLEVEL9K_INSTANT_PROMPT=quiet
```

## 1.2 requirements 설정

### 1.2.1 파일 경로에 requirements.txt 생성

```
transformers
datasets
```

생성 후 위의 내용 추가

### 1.2.2 requirements 설치

In [5]:
!pip3 install -r requirements.txt

## 1.3 import 라이브러리

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from datasets import load_dataset

## 1.4 데이터셋 가져오기

In [23]:
ds = load_dataset('imdb')
train_ds = ds['train']
test_ds  = ds['test']

## 1.5 checkpoint 설정

In [18]:
checkpoint = 'distilbert-base-uncased'

## 1.6 Tokenize

In [42]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(texts):
    return tokenizer(
        texts['text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

train_ds = train_ds.map(tokenize, batched = True)


Map: 100%|████████████████| 25000/25000 [00:03<00:00, 6520.41 examples/s]


In [44]:

# Select only the columns we need for training
train_ds = train_ds.remove_columns(['text'])  # Remove the original text
train_ds = train_ds.rename_column('label', 'labels')  # Rename 'label' to 'labels' as expected by the model

# Convert to PyTorch tensors
train_ds.set_format('torch')

# Create DataLoader for batched training
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_ds,
    batch_size=8,  # Adjust based on your GPU memory
    shuffle=True
)

In [45]:
train_ds[0]

{'labels': tensor(0),
 'input_ids': tensor([  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
          2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
          2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
          2657,  2008,  2012,  2034,  2009,  2001,  8243,  2011,  1057,  1012,
          1055,  1012,  8205,  2065,  2009,  2412,  2699,  2000,  4607,  2023,
          2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,  1000,
          6801,  1000,  1045,  2428,  2018,  2000,  2156,  2023,  2005,  2870,
          1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  1996,
          5436,  2003,  8857,  2105,  1037,  2402,  4467,  3689,  3076,  2315,
         14229,  2040,  4122,  2000,  4553,  2673,  2016,  2064,  2055,  2166,
          1012,  1999,  3327,  2016,  4122,  2000,  3579,  2014,  3086,  2015,
          2000,  2437,  2070,  4066,  1997,  4516,  2006,  2054,  1996,  2779,
         25430, 1

## 1.7 모델 정의

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
device

device(type='mps')

In [56]:
# 데이터셋을 1000개 선택 (50 -> 1000으로 증가)
small_train_ds = train_ds.select(range(1000))

# 검증용 데이터셋 200개 별도 분리
train_dataset = small_train_ds.select(range(800))
val_dataset = small_train_ds.select(range(800, 1000))

# 배치 사이즈 증가 (2 -> 8)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# 학습률 조정 (5e-5 -> 1e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# 훈련 루프 개선
for epoch in range(num_epochs):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0
    
    # tqdm으로 진행률 표시
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss
        
        # 정확도 계산
        preds = torch.argmax(outputs.logits, dim=-1)
        correct = (preds == batch['labels']).sum().item()
        total_correct += correct
        total_samples += batch['labels'].size(0)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    
    # 검증 단계 추가
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=-1)
            val_correct += (preds == batch['labels']).sum().item()
            val_total += batch['labels'].size(0)
    
    # 결과 출력
    train_acc = (total_correct / total_samples) * 100
    val_acc = (val_correct / val_total) * 100
    avg_loss = total_loss / len(train_dataloader)
    
    print(f"Epoch {epoch+1}:")
    print(f"Train Loss: {avg_loss:.4f} | Acc: {train_acc:.2f}%")
    print(f"Val Acc: {val_acc:.2f}%\n")

Epoch 1: 100%|█████████████████████████| 100/100 [00:56<00:00,  1.76it/s]


Epoch 1:
Train Loss: 0.0000 | Acc: 100.00%
Val Acc: 100.00%



Epoch 2: 100%|█████████████████████████| 100/100 [00:56<00:00,  1.77it/s]


Epoch 2:
Train Loss: 0.0000 | Acc: 100.00%
Val Acc: 100.00%



Epoch 3: 100%|█████████████████████████| 100/100 [00:56<00:00,  1.77it/s]


Epoch 3:
Train Loss: 0.0000 | Acc: 100.00%
Val Acc: 100.00%

